# 00 Setup

Configure your environment, understand the repo layout, and load sample data.


## Table of Contents
- [Environment bootstrap](#environment-bootstrap)
- [Verify API keys](#verify-api-keys)
- [Load sample data](#load-sample-data)
- [Checkpoints](#checkpoints)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Foundations notebooks build the intuition that prevents the most common mistakes in economic ML:
- leaking future information,
- evaluating with the wrong split strategy,
- over-interpreting coefficients.


## Prerequisites (Quick Self-Check)
- Comfort with basic Python + pandas (reading CSVs, making plots).
- Willingness to write short interpretation notes as you go.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Using random splits on time series.
- Assuming correlation implies causation.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/00_foundations/00_setup.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/00_foundations/00_setup.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Primer: Paths, files, and environment variables (how this repo stays reproducible)

You will see a few patterns repeatedly in notebooks and scripts.

### Environment variables (API keys)

Environment variables are key/value settings provided by your shell to Python.
This repo uses them for API keys:
- `FRED_API_KEY`
- `CENSUS_API_KEY` (optional)

```python
import os

fred_key = os.getenv("FRED_API_KEY")
print("FRED key set?", fred_key is not None)
```

If you set a key in a terminal, restart the Jupyter kernel so Python sees it.

### Paths (why `pathlib.Path` is the default)

Use `Path` to build OS-safe file paths:

```python
from pathlib import Path

p = Path("data") / "sample" / "macro_quarterly_sample.csv"
print(p, "exists?", p.exists())
```

### Repo bootstrap variables (defined in every notebook)

The notebook bootstrap cell defines:
- `PROJECT_ROOT` (repo root)
- `DATA_DIR`, `RAW_DIR`, `PROCESSED_DIR`, `SAMPLE_DIR`

Prefer these over hard-coded relative paths.

### Sample vs processed data (offline-first)

Most notebooks follow this pattern:
1) try `data/processed/*` (real pipeline output)
2) fall back to `data/sample/*` (small offline dataset)

This keeps notebooks runnable without network access.

### Common “file not found” fixes

- Print the path and check `.exists()`
- Print current working directory:
  - `import os; print(os.getcwd())`
- Start Jupyter from the repo root (so bootstrap can find `src/` and `docs/`)


<a id="verify-api-keys"></a>
## Verify API keys

In this project you will use two APIs:
- **FRED** (macro time series)
- **US Census ACS** (micro cross-sectional data)

We read API keys from environment variables:
- `FRED_API_KEY`
- `CENSUS_API_KEY` (optional; many endpoints work without it)

Your task: confirm your notebook process can see these variables.



### Your Turn (1): Read env vars safely


In [ ]:
import os

# TODO: Read env vars with os.getenv
fred_key = os.getenv('FRED_API_KEY')
census_key = os.getenv('CENSUS_API_KEY')

# TODO: Print whether each key is set (do NOT print the full key)
# Hint: show the first 4 chars only if present
...



### Your Turn (2): Confirm your data folders exist


In [ ]:
# TODO: Print the important directories defined by the bootstrap cell.
# Confirm they exist (or create them where appropriate).
print('PROJECT_ROOT:', PROJECT_ROOT)
print('DATA_DIR:', DATA_DIR)
print('RAW_DIR:', RAW_DIR)
print('PROCESSED_DIR:', PROCESSED_DIR)
print('SAMPLE_DIR:', SAMPLE_DIR)

# TODO: Create RAW_DIR and PROCESSED_DIR if they don't exist
...



<a id="load-sample-data"></a>
## Load sample data

This repo includes small sample datasets under `data/sample/` so you can work offline.
In most notebooks you'll:
1) try to load a real dataset from `data/processed/`
2) if it doesn't exist yet, load a sample from `data/sample/`

Your task: load the macro quarterly sample and inspect the schema.



### Your Turn (1): Load a sample dataset


In [ ]:
import pandas as pd

# TODO: Load data/sample/macro_quarterly_sample.csv
# Hint: use index_col=0 and parse_dates=True
df = ...

# TODO: Inspect the data
# - df.shape
# - df.columns
# - df.dtypes
...



### Your Turn (2): Quick plot


In [ ]:
import matplotlib.pyplot as plt

# TODO: Pick 1-2 columns to plot over time (e.g., GDP growth + recession label)
# Hint: df[['col1','col2']].plot(subplots=True)
...



<a id="checkpoints"></a>
## Checkpoints

These are the kinds of checks you will repeat in every notebook.
If you build this habit early, debugging becomes dramatically easier.



### Checkpoint (data sanity)


In [ ]:
# TODO: Replace column names below with real ones from df
# Example expected columns in macro_quarterly: gdp_growth_qoq, recession, target_recession_next_q

# 1) Index checks
assert df.index.is_monotonic_increasing
assert df.index.inferred_type in {'datetime64', 'datetime64tz'}

# 2) Shape checks
assert df.shape[0] > 20
assert df.shape[1] >= 3

# 3) Missingness checks (you may allow some NaNs early due to lags)
print(df.isna().sum().sort_values(ascending=False).head(10))

# TODO: Write 2 sentences: what does each check protect you from?
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run a quick sanity check on any DataFrame/Series you created in this notebook.
# Example (adjust variable names):
# assert df.index.is_monotonic_increasing
# assert df.isna().sum().sum() == 0
#
# TODO: Write 2-3 sentences:
# - What would leakage look like in YOUR code?
# - How would you detect it?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Environment bootstrap</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_setup — Environment bootstrap
print(PROJECT_ROOT)
print(DATA_DIR)
```

</details>

<details><summary>Solution: Verify API keys</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_setup — Verify API keys
import os
print('FRED_API_KEY set?', bool(os.getenv('FRED_API_KEY')))
print('CENSUS_API_KEY set?', bool(os.getenv('CENSUS_API_KEY')))
```

</details>

<details><summary>Solution: Load sample data</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_setup — Load sample data
import pandas as pd
df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)
df.head()
```

</details>

<details><summary>Solution: Checkpoints</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_setup — Checkpoints
assert df.index.is_monotonic_increasing
assert 'target_recession_next_q' in df.columns
print('ok')
```

</details>

